In [ ]:
# Connect to google drive to fetch preprocessed data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import cv2
import gc
from sklearn.model_selection import KFold
from google.colab.patches import cv2_imshow

In [ ]:
# Load Correctly Masked Faces
correctly_features=np.load("/content/drive/MyDrive/MSProjectData/Dataset3/Correct/correct_features.npz")
# Load Incorrectly Masked Faces
incorrect_features=np.load("/content/drive/MyDrive/MSProjectData/Dataset3/Incorrect/incorrect_features.npz")

In [ ]:
# Extract Correctly Masked Features from npz 
correctly_masked=correctly_features['arr_0']
correctly_masked=correctly_masked/255
# Extract Incorrectly Masked Features from npz
incorrectly_masked=incorrect_features['arr_0']
incorrectly_masked=incorrectly_masked/255

In [ ]:
del correctly_features
del incorrect_features

In [ ]:
# Preparing Training And Testing Data for 5-fold cross validation
trainingCorrectlyMasked=[]
testingCorrectlyMasked=[]
trainingIncorrectlyMasked=[]
testingIncorrectlyMasked=[]
kfold=KFold(n_splits=5,shuffle=True)
for train,test in kfold.split(range(len(correctly_masked))):
  trainingCorrectlyMasked.append(correctly_masked[train])
  testingCorrectlyMasked.append(correctly_masked[test])
for train,test in kfold.split(range(len(incorrectly_masked))):
  trainingIncorrectlyMasked.append(incorrectly_masked[train])
  testingIncorrectlyMasked.append(incorrectly_masked[test])
trainingCorrectlyMasked=np.asarray(trainingCorrectlyMasked)
trainingIncorrectlyMasked=np.asarray(trainingIncorrectlyMasked)
testingCorrectlyMasked=np.asarray(testingCorrectlyMasked)
testingIncorrectlyMasked=np.asarray(testingIncorrectlyMasked)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
print(trainingCorrectlyMasked.shape,trainingIncorrectlyMasked.shape,testingCorrectlyMasked.shape,testingIncorrectlyMasked.shape)

(5,) (5,) (5,) (5,)


In [ ]:
# Import Tesortflow and Keras libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv1D,Conv2D,Flatten,MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
A=[0,3,6,9,12,15,18,21,24,27,30,33] 
B=[1,4,7,10,13,16,19,22,25,28,31,34]
C=[2,5,8,11,14,17,20,23,26,29,32,35] 
D=[36,37,38] 
customMadeIncorrectLabels=[]
for i in range(39): # Because there are total 39 incorrectly masked images per subject
  if i in A:
    customMadeIncorrectLabels.append("Nose_Uncovered")
  elif i in B:
    customMadeIncorrectLabels.append("NoseMouth_Uncovered")
  elif i in C:
    customMadeIncorrectLabels.append("Chin_Uncovered")
  elif i in D:
    customMadeIncorrectLabels.append("No_Mask")
  else:
    print("Error",i)

In [ ]:
accuracy=[]
correctlyAccuracy=[]
incorrectlyAccuracy=[]
countNose_Uncovered=0
countNoseMouth_Uncovered=0
countChin_Uncovered=0
countNo_Mask=0
for i in range(0,5):
  # Preparing Training And Testing Set
  customMadeIncorrectLabels=customMadeIncorrectLabels*len(testingIncorrectlyMasked[i])
  trainCorrect=np.reshape(trainingCorrectlyMasked[i],(trainingCorrectlyMasked[i].shape[0]*trainingCorrectlyMasked[i].shape[1],trainingCorrectlyMasked[i].shape[2],trainingCorrectlyMasked[i].shape[3],trainingCorrectlyMasked[i].shape[4]))
  testCorrect=np.reshape(testingCorrectlyMasked[i],(testingCorrectlyMasked[i].shape[0]*testingCorrectlyMasked[i].shape[1],testingCorrectlyMasked[i].shape[2],testingCorrectlyMasked[i].shape[3],testingCorrectlyMasked[i].shape[4]))
  trainIncorrect=np.reshape(trainingIncorrectlyMasked[i],(trainingIncorrectlyMasked[i].shape[0]*trainingIncorrectlyMasked[i].shape[1],trainingIncorrectlyMasked[i].shape[2],trainingIncorrectlyMasked[i].shape[3],trainingIncorrectlyMasked[i].shape[4]))
  testIncorrect=np.reshape(testingIncorrectlyMasked[i],(testingIncorrectlyMasked[i].shape[0]*testingIncorrectlyMasked[i].shape[1],testingIncorrectlyMasked[i].shape[2],testingIncorrectlyMasked[i].shape[3],testingIncorrectlyMasked[i].shape[4]))
  x_train=np.concatenate((trainCorrect,trainIncorrect),axis=0)
  x_test=np.concatenate((testCorrect,testIncorrect),axis=0)
  y_train=np.concatenate((np.zeros(len(trainCorrect)),np.ones(len(trainIncorrect))),axis=0)
  y_test=np.concatenate((np.zeros(len(testCorrect)),np.ones(len(testIncorrect))),axis=0)
  print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
  # CNN Model
  model=Sequential()
  model.add(Conv2D(filters=128,kernel_size=(5,5),activation='relu',input_shape=x_train[0].shape))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(units=128,activation='relu'))
  model.add(Dense(units=32,activation='relu'))
  model.add(Dense(units=1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  model.summary()
  class MyClass(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      # Stop the model if accuracy reaches 99.5%
      if logs.get('accuracy') > 0.995:
        self.model.stop_training=True
  model.fit(x_train,y_train,epochs=500,batch_size=128,shuffle=True,callbacks=[MyClass()])
  accuracy.append(model.evaluate(x_test,y_test)[1])
  correctlyAccuracy.append(model.evaluate(testCorrect,np.zeros(len(testCorrect)))[1])
  incorrectlyAccuracy.append(model.evaluate(testIncorrect,np.ones(len(testIncorrect)))[1])
  incorrectLabels=model.predict_classes(testIncorrect)
  for j in range(len(incorrectLabels)):
    if customMadeIncorrectLabels[j]=="Nose_Uncovered":
      if incorrectLabels[j]==1:
        countNose_Uncovered=countNose_Uncovered+1
    elif customMadeIncorrectLabels[j]=="NoseMouth_Uncovered":
      if incorrectLabels[j]==1:
        countNoseMouth_Uncovered=countNoseMouth_Uncovered+1
    elif customMadeIncorrectLabels[j]=="Chin_Uncovered":
      if incorrectLabels[j]==1:
        countChin_Uncovered=countChin_Uncovered+1
    elif customMadeIncorrectLabels[j]=="No_Mask":
      if incorrectLabels[j]==1:
        countNo_Mask=countNo_Mask+1
    else:
      print("Error")

(4725, 16, 16, 1) (4725,) (1200, 16, 16, 1) (1200,)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 12, 12, 128)       3328      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 4, 4, 32)          36896     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 2, 2, 32)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               16512     
_________________________________________________________________
de

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Epoch 1/500
37/37 [==============================] - 1s 3ms/step - loss: 0.6744 - accuracy: 0.6202
Epoch 2/500
37/37 [==============================] - 0s 3ms/step - loss: 0.4346 - accuracy: 0.8151
Epoch 3/500
37/37 [==============================] - 0s 3ms/step - loss: 0.1654 - accuracy: 0.9359
Epoch 4/500
37/37 [==============================] - 0s 3ms/step - loss: 0.1219 - accuracy: 0.9556
Epoch 5/500
37/37 [==============================] - 0s 3ms/step - loss: 0.1005 - accuracy: 0.9649
Epoch 6/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0816 - accuracy: 0.9738
Epoch 7/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0665 - accuracy: 0.9758
Epoch 8/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0606 - accuracy: 0.9804
Epoch 9/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0559 - accuracy: 0.9840
Epoch 10/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0468 - accuracy: 0.9854
Epoch 11/

In [ ]:
print("Average Accuracy:",sum(accuracy)/len(accuracy)*100)
print("\n")
totalTrainingSubjects=16
totalFold=5
print("Average Accuracy for Correctly Masked Faces:",sum(correctlyAccuracy)/len(correctlyAccuracy)*100)
print("Average Accuract for Incorrectly Masked only Chin Covered:",sum(incorrectlyAccuracy)/len(incorrectlyAccuracy)*100)
print("\tIncorrectly Masked: Nose Open Accuracy",countNose_Uncovered/(totalTrainingSubjects*12*totalFold)*100)
print("\tIncorrectly Masked: Nose Mouth Open Accuracy",countNoseMouth_Uncovered/(totalTrainingSubjects*12*totalFold)*100)
print("\tIncorrectly Masked: Chin Open Accuracy",countChin_Uncovered/(totalTrainingSubjects*12*totalFold)*100)
print("\tIncorrectly Masked: No Mask Accuracy",countNo_Mask/(totalTrainingSubjects*3*totalFold)*100)

Average Accuracy: 98.06333422660828


Average Accuracy for Correctly Masked Faces: 98.07175993919373
Average Accuract for Incorrectly Masked only Chin Covered: 98.05555582046509
	Incorrectly Masked: Nose Open Accuracy 96.04166666666667
	Incorrectly Masked: Nose Mouth Open Accuracy 98.54166666666667
	Incorrectly Masked: Chin Open Accuracy 95.625
	Incorrectly Masked: No Mask Accuracy 97.91666666666666


In [ ]:
model.save("/content/drive/MyDrive/MSProjectData/Dataset3/SCFaceModel.h5")